In [1]:
import os
import shutil
import random
import pandas as pd
from src.data import download_images

c:\Users\wolfg\OneDrive\Documents\CivitAI\Project Work\research\multimodalCheck\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get Data and sort it

In [7]:
# data = pd.read_csv('data/reviewed_data_5_13_24.csv') ##We can use this for training
data = pd.read_csv('data/justin_dataset_may_models.csv').rename(columns={'new_url': 'download_url'}) ##This we can use to bolster the R rated dataset


In [8]:
data.groupby('label')['label'].count()

label
PG      1663
PG13    2173
R        479
X        188
XXX      194
Name: label, dtype: int64

In [9]:
# Count the number of rows per label
label_counts = data.groupby('label')['label'].count()

# Get the minimum count among all labels
min_count = label_counts.min()

# Sample equal number of rows per label
balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

balanced_data.groupby('label')['label'].count()

C:\Users\wolfg\AppData\Local\Temp\ipykernel_17768\3553324668.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
PG      188
PG13    188
R       188
X       188
XXX     188
Name: label, dtype: int64

In [10]:
balanced_data.head()

,download_url,prompt,tags,prediction,label,model_preds,baseresNet18,baseresNet50,promptBert,promptTagBert,resNet18CV,resNet50CV,roberta,5_model_prediction
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"<lora:XL_dohna dohna style:1>,dohna dohna styl...","virtual youtuber, twintails, solo, small breas...",PG,PG,"{'baseresNet18': 'PG', 'baseresNet50': 'PG-13'...",PG,PG-13,PG,PG,R,PG,PG13,PG
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"1girl (plastic), teen, gothic, pale skin yello...","book, comics, publication, adult, very long ha...",PG,PG,"{'baseresNet18': 'PG', 'baseresNet50': 'PG', '...",PG,PG,PG,PG,PG,PG,PG,PG
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,x1a_n0texist as a dark sorceress in a dress st...,"face, head, person, photography, portrait, adu...",PG,PG,"{'baseresNet18': 'PG', 'baseresNet50': 'PG', '...",PG,PG,PG,PG,PG,PG,PG,PG
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"Merger_marupe_sash,\n(African tribe dancing we...","toy, hula, costume, clothing, person, revealin...",PG,PG,"{'baseresNet18': 'R', 'baseresNet50': 'PG', 'p...",R,PG,PG,PG,PG,PG,PG,PG
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"d1pw3dd1ngR3d, bare shoulders, standing, full ...","revealing clothes, woman, person, bride, adult...",PG,PG,"{'baseresNet18': 'PG', 'baseresNet50': 'PG', '...",PG,PG,PG,PG,PG,PG,PG,PG


In [12]:
data = {}

# Iterate over the DataFrame
for index, row in balanced_data.iterrows():
    data[index] = {
        "url": row['download_url'], 
        "prompt": row['prompt'],
        "prompt_and_tags": f"{row['prompt']} {row['tags']}",
        "label": row['label']
    }


In [13]:
output = './data/eval_data/'
download_images(data, output)

10138589.jpg
5006783.jpg
5011357.jpg
5009395.jpg
8078872.jpg
8004825.jpg
5008150.jpg
8078939.jpg
5017112.jpg
6152787.jpg
6693384.jpg
5018302.jpg
8004666.jpg
5021954.jpg
8078419.jpg
6913131.jpg
5008098.jpg
6910175.jpg
7034088.jpg
5015549.jpg
5005164.jpg
10135162.jpg
Error processing image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/056617e9-9f96-4e5a-871a-e17fb6065d23/width=450/10135162.jpg: write() argument must be str, not float
5002826.jpg
5008474.jpg
5001306.jpg
5007915.jpg
8078711.jpg
6262182.jpg
7034499.jpg
5011175.jpg
6693393.jpg
5002150.jpg
6912021.jpg
5007650.jpg
8078296.jpg
Error processing image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/8ff6abce-9cd3-4730-b6a9-e6886eac8e1d/width=450/8078296.jpg: write() argument must be str, not float
5000470.jpg
5015000.jpg
Error processing image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/8b432109-15cc-4e17-a0e0-6651cb3b4b32/width=450/5015000.jpg: write() argument must be str, not float
8004826.jpg
10143148.j

In [14]:
for i in os.listdir(output):
    path = os.path.join(output, i)
    print(i, len([i for i in os.listdir(path) if i.endswith('.txt') and 'tags' not in i]))
    print(i, len([i for i in os.listdir(path) if i.endswith('.jpg')]))

PG 188
PG 188
PG13 188
PG13 188
R 188
R 188
X 188
X 188
XXX 188
XXX 188


## Use loop for training data

In [15]:


# # Define the source directory containing class-specific folders
# source_dir = './data/training_data/'

# # Define the output directories for training and validation
# output_train_dir = './data/datasets/may/train'
# output_val_dir = './data/datasets/may/val'

# # Create the output directories if they don't exist
# os.makedirs(output_train_dir, exist_ok=True)
# os.makedirs(output_val_dir, exist_ok=True)

# # Iterate over each class-specific folder
# for class_name in os.listdir(source_dir):
#     class_path = os.path.join(source_dir, class_name)
    
#     if os.path.isdir(class_path):
#         # List of all .txt files (excluding those with 'tags' in their name)
#         txt_files = [f for f in os.listdir(class_path) if f.endswith('.txt') and 'tags' not in f]
        
#         num_files = len(txt_files)
#         train_split = int(num_files * 0.85)
        
#         # Create lists of indices for training and validation sets
#         shuffled_indices = list(range(num_files))
#         random.shuffle(shuffled_indices)
#         train_indices = shuffled_indices[:train_split]
#         val_indices = shuffled_indices[train_split:]

#         print(f"Class '{class_name}':")
#         print(f"  Train indices: {len(train_indices)}")
#         print(f"  Val indices: {len(val_indices)}")

#         # Copy files into 'train' and 'val' directories
#         for i in train_indices:
#             txt_file = txt_files[i]
#             img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
#             # Source paths
#             src_txt_path = os.path.join(class_path, txt_file)
#             src_img_path = os.path.join(class_path, img_file)
            
#             # Destination paths
#             dest_txt_path = os.path.join(output_train_dir, class_name, txt_file)
#             dest_img_path = os.path.join(output_train_dir, class_name, img_file)
            
#             # Copy the files
#             os.makedirs(os.path.join(output_train_dir, class_name), exist_ok=True)
#             shutil.copy2(src_txt_path, dest_txt_path)
#             if os.path.exists(src_img_path):
#                 shutil.copy2(src_img_path, dest_img_path)

#         for i in val_indices:
#             txt_file = txt_files[i]
#             img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
#             # Source paths
#             src_txt_path = os.path.join(class_path, txt_file)
#             src_img_path = os.path.join(class_path, img_file)
            
#             # Destination paths
#             dest_txt_path = os.path.join(output_val_dir, class_name, txt_file)
#             dest_img_path = os.path.join(output_val_dir, class_name, img_file)
            
#             os.makedirs(os.path.join(output_val_dir, class_name), exist_ok=True)
            
#             # Copy the files
#             shutil.copy2(src_txt_path, dest_txt_path)
#             if os.path.exists(src_img_path):
#                 shutil.copy2(src_img_path, dest_img_path)

# print("File copying complete!")

## User loop for eval

In [ ]:


# Define the source directory containing class-specific folders
print(output_dir)

# Define the output directories for training and validation
output_train_dir = './data/datasets/may/train'
output_val_dir = './data/datasets/may/val'

# Create the output directories if they don't exist
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_val_dir, exist_ok=True)

# Iterate over each class-specific folder
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    
    if os.path.isdir(class_path):
        # List of all .txt files (excluding those with 'tags' in their name)
        txt_files = [f for f in os.listdir(class_path) if f.endswith('.txt') and 'tags' not in f]
        
        num_files = len(txt_files)
        train_split = int(num_files * 0.85)
        
        # Create lists of indices for training and validation sets
        shuffled_indices = list(range(num_files))
        random.shuffle(shuffled_indices)
        train_indices = shuffled_indices[:train_split]
        val_indices = shuffled_indices[train_split:]

        print(f"Class '{class_name}':")
        print(f"  Train indices: {len(train_indices)}")
        print(f"  Val indices: {len(val_indices)}")

        # Copy files into 'train' and 'val' directories
        for i in train_indices:
            txt_file = txt_files[i]
            img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            
            # Destination paths
            dest_txt_path = os.path.join(output_train_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_train_dir, class_name, img_file)
            
            # Copy the files
            os.makedirs(os.path.join(output_train_dir, class_name), exist_ok=True)
            shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)

        for i in val_indices:
            txt_file = txt_files[i]
            img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            
            # Destination paths
            dest_txt_path = os.path.join(output_val_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_val_dir, class_name, img_file)
            
            os.makedirs(os.path.join(output_val_dir, class_name), exist_ok=True)
            
            # Copy the files
            shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)

print("File copying complete!")